# To extract tables from arxiv PDFs
Started: Oct 9th- Last edit: Oct 11th
By Shooby

In [1]:
from img2table.document import PDF
from img2table.ocr import TesseractOCR
#from img2table.ocr import EasyOCR
import pandas as pd
import os


In [2]:
def remove_enter_row(row):
    ''' This module is a preprocessing to further split the table 
    if there are \n left in the data'''
    
    try:
        numenters = row.values[0].count('\n')
        for i in range(numenters):
            new_row = row.copy()
            for col, value in row.items():
                split_values = value.split('\n')
                new_row[col] = split_values[1]
                row[col]=split_values[0]
        return[row,new_row]
    except:
        return [row]
        
def remove_character(string, character):
  if character not in string:
    return string
  return string.replace(character, "")
    
def cleansave(extracted_tables,dir):
    for page, tables in extracted_tables.items():
        for idx, table in enumerate(tables):
            print('-----------------------------------------------------------------------')
            print('Title:'+str(table.title))
            if table.title:
                t = table.title
                if ' ' in t:
                    t = t.replace(' ','_')
                t = remove_character(t,'.')
                t = remove_character(t,"\\")
                t = remove_character(t,"/")
                t = remove_character(t,",")
                tablename = t+'.csv'
            else:
                tablename = 'table'+str(idx)+'_page'+str(page)+'.csv'
            tb = table.df
            processed_rows = [remove_enter_row(row) for index, row in tb.iterrows()]
            flattened_rows = [item for sublist in processed_rows for item in sublist]
            result_df = pd.DataFrame(flattened_rows, columns=tb.columns)
            dfp = result_df.fillna('-')
            dfp.to_csv(dir+'/'+tablename, index=False)
            print('-----------------------------------------------------------------------')


def pdf2csv(pdffile):
    pdf = PDF(src=pdffile)
    ocr = TesseractOCR(lang="eng")
    #easyocr = EasyOCR(lang=["en"])
    extracted_tables = pdf.extract_tables(ocr=ocr,
                                          implicit_rows=True,
                                          borderless_tables=False,
                                          min_confidence=80)
    dirname = pdffile.replace('.pdf','_tables')
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        
    cleansave(extracted_tables,dirname)

In [3]:
pdf2csv('2110.06945.pdf')

/bin/sh: 1: tesseract: not found


OSError: Tesseract not found in environment. Check variables and PATH

In [ ]:
pdf = PDF(src='20')
ocr = TesseractOCR(lang="eng")
extracted_tables = pdf.extract_tables(ocr=easyocr,implicit_rows=True,borderless_tables=False,min_confidence=30)
for page, tables in extracted_tables.items():
    for idx, table in enumerate(tables):
        print(table)
